In [1]:
import pandas as pd
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np
import ast


nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')




[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>


False

## Loading Movie Meta Data

In [2]:
path = "../data"
movie_meta_file = "/movies_metadata.csv"
df_meta = pd.read_csv(path+movie_meta_file)

/var/folders/5w/zg23p5bd3bnc86d75_pkn99m0000gn/T/ipykernel_98629/4212106273.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_meta = pd.read_csv(path+movie_meta_file)


In [3]:

df_meta = df_meta.drop([19730, 29503, 35587])
df_meta = df_meta.set_index(df_meta['id'].str.strip().replace(',','').astype(int))


# Extract important features

meta_features = ['genres', 'imdb_id', 'original_language', 'revenue', 'release_date', 'spoken_languages', 'title', 'vote_average', 'vote_count', 'overview']
meta = df_meta[meta_features]

## Credits

In [4]:
# credits 

credits = pd.read_csv(path+ "/credits.csv")

credits

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


In [6]:
# Create col for directors
credits['Director'] = credits['crew'].apply(lambda x:[dct['name'] for dct in ast.literal_eval(x) if dct['job'] == 'Director'])

df_credits = credits.set_index('id')

In [ ]:
df_credits

In [9]:
# Keywords 

df_keywords = pd.read_csv(path + "/keywords.csv")

df_keywords = df_keywords.set_index('id')


In [10]:
# Merging datasets
df_key_credit = df_keywords.merge(df_credits, left_index=True, right_on='id')
df = df_key_credit.merge(meta, left_index=True, right_on='id')


## Data Cleaning

In [16]:
df['genres'] = df['genres'].apply(lambda x: [i['name'] for i in ast.literal_eval(x)])

df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)


df['keywords'] = df['keywords'].apply(lambda x: [i['name'] for i in eval(x)])
df['keywords'] = df['keywords'].apply(lambda x: ' '.join([i.replace(" ", "") for i in x]))

# extract the overview
df['overview'] = df['overview'].fillna('')

### Converting final dataframe to csv file

In [15]:
df.to_csv("final_movie.csv",index=False)

## NLP - Text processing

In [11]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/andresrivera/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/andresrivera/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/andresrivera/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
def preprocess_overview(text):

    if not isinstance(text, str):
        return []
    
    # tokenize the text
    tokens = word_tokenize(text)

     # Convert to lowercase and remove non-alphanumeric characters
    words = [word.lower() for word in tokens if word.isalnum()]
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word not in stop_words]
    
    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]
    
    return lemmatized_words




df["overview"].apply(preprocess_overview)

In [14]:
# Preprocessing text data
def extract_keywords(row):
    keyword_list = ast.literal_eval(row['keywords'])
    keywords = [kw['name'] for kw in keyword_list]
    directors = row['directors']
    #overview_keywords = preprocess_overview(row['overview'])
    genres = row['genres'].lower().split(' ')

    return genres + keywords + directors 


In [ ]:

df_movies = pd.DataFrame()


# Genres
df_movies['genres'] = df_movies['genres'].apply(lambda x: ' '.join([i.replace(" ", "") for i in x]))
df_movies['keywords'] = df['keywords']
df_movies['directors'] = df['Director']

# Text analysis

df_movies['combined_keywords'] = df_movies.apply(extract_keywords, axis=1)

df_movies['title'] = df['title']

In [ ]:
# removing rows with no genres

mask = df_movies['genres'].str.strip() != ""

df_movies = df_movies[mask]

df_movies.genres.value_counts(dropna=False)

In [136]:
def join_keywords(keywords_list):
    return ' '.join(keywords_list)

df_movies['combined_keywords_str'] = df_movies['combined_keywords'].apply(join_keywords)

/var/folders/5w/zg23p5bd3bnc86d75_pkn99m0000gn/T/ipykernel_84307/3582924666.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movies['combined_keywords_str'] = df_movies['combined_keywords'].apply(join_keywords)


In [ ]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df_movies['combined_keywords_str'])

In [143]:
recommended_movies = recommend_movies("Inception", 10)
print(recommended_movies)

id
191185           My Sweet Orange Tree
383146                        Curumim
40823              The Man Who Copied
64397     Basic Sanitation, the Movie
29275                           Anita
72560              Federal Bank Heist
81022                          Widows
78705                    Live-In Maid
113329           Artificial Paradises
222772                   Lion's Heart
Name: title, dtype: object


In [139]:
def recommend_movies(title, num_recommendations=10):
    # Get the index of the movie with the given title
    movie_index = df_movies[df_movies['title'] == title].index[0]

    # Get the list of cosine similarity scores for the given movie
    similarity_scores = list(enumerate(cosine_sim[movie_index]))

    # Sort the scores in descending order
    similarity_scores_sorted = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    # Get the indices of the most similar movies (excluding the input movie itself)
    similar_movie_indices = [score[0] for score in similarity_scores_sorted[1:num_recommendations + 1]]

    # Return the titles of the most similar movies
    return df_movies['title'].iloc[similar_movie_indices]

In [23]:
# create an empty DataFrame
df_movies = pd.DataFrame()

# extract the keywords
df_movies['keywords'] = df['keywords'].apply(lambda x: [i['name'] for i in eval(x)])
df_movies['keywords'] = df_movies['keywords'].apply(lambda x: ' '.join([i.replace(" ", "") for i in x]))

# extract the overview
df_movies['overview'] = df['overview'].fillna('')

# extract the release year 
df_movies['release_date'] = pd.to_datetime(df['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

# extract the actors
df_movies['cast'] = df['cast'].apply(lambda x: [i['name'] for i in eval(x)])
df_movies['cast'] = df_movies['cast'].apply(lambda x: ' '.join([i.replace(" ", "") for i in x]))

# extract genres
df_movies['genres'] = df['genres'].apply(lambda x: [i['name'] for i in eval(x)])
df_movies['genres'] = df_movies['genres'].apply(lambda x: ' '.join([i.replace(" ", "") for i in x]))

# add the title
df_movies['title'] = df['title']

# merge fields into a tag field
df_movies['tags'] = df_movies['keywords'] + df_movies['cast']+' '+df_movies['genres']+' '+df_movies['release_date']

# drop records with empty tags and dublicates
df_movies.drop(df_movies[df_movies['tags']==''].index, inplace=True)
df_movies.drop_duplicates(inplace=True)

# add a fresh index to the dataframe, which we will later use when refering to items in a vector matrix
df_movies['new_id'] = range(0, len(df_movies))

# Reduce the data to relevant columns
df_movies = df_movies[['new_id', 'title', 'tags']]

# display the data
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.expand_frame_repr', False)
print(df_movies.shape)
df_movies.head(5)

(45432, 3)


,new_id,title,tags
id,,,
862,0,Toy Story,jealousy toy boy friendship friends rivalry boynextdoor newtoy toycomestolifeTomHanks TimAllen DonRickles JimVarney WallaceShawn JohnRatzenberger AnniePotts JohnMorris ErikvonDetten LaurieMetcalf R.LeeErmey SarahFreeman PennJillette Animation Comedy Family 1995
8844,1,Jumanji,boardgame disappearance basedonchildren'sbook newhome recluse giantinsectRobinWilliams JonathanHyde KirstenDunst BradleyPierce BonnieHunt BebeNeuwirth DavidAlanGrier PatriciaClarkson AdamHann-Byrd LauraBellBundy JamesHandy GillianBarber BrandonObray CyrusThiedeke GaryJosephThorup LeonardZola LloydBerry MalcolmStewart AnnabelKershaw DarrylHenriques RobynDriscoll PeterBryant SarahGilson FloricaVlad JuneLion BrendaLockmuller Adventure Fantasy Family 1995
15602,2,Grumpier Old Men,fishing bestfriend duringcreditsstinger oldmenWalterMatthau JackLemmon Ann-Margret SophiaLoren DarylHannah BurgessMeredith KevinPollak Romance Comedy 1995
31357,3,Waiting to Exhale,basedonnovel interracialrelationship singlemother divorce chickflickWhitneyHouston AngelaBassett LorettaDevine LelaRochon GregoryHines DennisHaysbert MichaelBeach MykeltiWilliamson LamontJohnson WesleySnipes Comedy Drama Romance 1995
11862,4,Father of the Bride Part II,baby midlifecrisis confidence aging daughter motherdaughterrelationship pregnancy contraception gynecologistSteveMartin DianeKeaton MartinShort KimberlyWilliams-Paisley GeorgeNewbern KieranCulkin BDWong PeterMichaelGoetz KateMcGregor-Stewart JaneAdams EugeneLevy LoriAlan Comedy 1995


In [24]:
# set a custom stop list from nltk
stop = list(stopwords.words('english'))

# create the tfid vectorizer, alternatively you can also use countVectorizer
tfidf = TfidfVectorizer(max_features=5000, analyzer='word', stop_words=stop)
vectorized_data = tfidf.fit_transform(df_movies['tags'])
count_matrix = pd.DataFrame(vectorized_data.toarray(), index=df_movies['tags'].index.tolist())
print(count_matrix)


        0     1     2     3     4     5     6     7     8     9     ...  4990  4991  4992  4993  4994  4995  4996  4997  4998  4999
862      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
8844     0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
15602    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
31357    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
11862    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
...      ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...
439050   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.

In [26]:
# compute the cosine similarity matrixS
similarity = cosine_similarity(reduced_data)
similarity

: 

: 

In [138]:
cosine_sim = cosine_similarity(tfidf_matrix)